Load data

In [1]:
import os
import pandas as pd
import glob
from typing import Tuple, List
import numpy as np

In [2]:
filenames = []
path_to_data_folder = os.path.join(os.getcwd(), "data")
print(os.path.join(path_to_data_folder, '*.txt'))
for f in glob.glob(os.path.join(path_to_data_folder, '*.txt')):
    print(f)
    variable_name = os.path.splitext(os.path.basename(f))[0]
    print(type(variable_name))
    filenames.append(variable_name)
    globals()[variable_name] = pd.read_csv(f, delimiter='\t')
    globals()[variable_name].columns = ['Time[s]','VGRF-left1', 'VGRF-left2','VGRF-left3','VGRF-left4','VGRF-left5','VGRF-left6','VGRF-left7','VGRF-left8','VGRF-right1', 'VGRF-right2','VGRF-right3','VGRF-right4','VGRF-right5','VGRF-right6','VGRF-right7','VGRF-right8','Total Force-left','Total Force-right']
# print(filenames)    

C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\*.txt
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaCo01_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaCo02_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaCo02_02.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaCo03_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaCo03_02.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaCo04_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaCo04_02.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaCo05_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaCo05_02.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaCo06_01.txt
<class 'str'>
C:\Users\k

C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt26_02.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt26_10.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt27_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt27_02.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt27_10.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt28_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt28_02.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt28_10.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt29_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt29_02.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\GaPt29_10.txt
<c

C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt12_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt13_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt14_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt15_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt15_02.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt15_03.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt15_04.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt15_05.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt15_06.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt15_07.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\JuPt16_01.txt
<c

C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt09_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt10_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt12_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt13_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt14_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt15_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt16_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt17_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt18_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt19_01.txt
<class 'str'>
C:\Users\kajot\PycharmProjects\Gait-in-Parkinson-s-Disease\data\SiPt20_01.txt
<c

In [3]:
def divide_dataset_to_Co_and_Pt(path_to_data_folder: str) -> Tuple[List[str], List[str]]:
    """

    Parameters
    ----------
    path_to_data_folder: str
        path to data folder

    Returns
    -------
    file_names_Co: List[str]
        list contains names of files containing Co in name
    file_names_Pt: List[str]
        list contains names of files containing Pt in name
    """

    file_names = os.listdir(path_to_data_folder)
    file_names_Co = [i for i in file_names if 'Co' in i]
    file_names_Pt = [i for i in file_names if 'Pt' in i]
    return file_names_Co, file_names_Pt

In [4]:
file_names_Co, file_names_Pt = divide_dataset_to_Co_and_Pt(path_to_data_folder)

Gait Phase Detection


In [5]:
import scipy.signal
import matplotlib.pyplot as plt

In [6]:
def remove_spikes(signal: pd.core.frame.DataFrame) -> List[float]:
    """
    Remove spikes within swing phase

    Parameters
    ----------
    signal: pd.core.frame.DataFrame
        data frame contains dynamic center od pressure during gait

    Returns
    -------
    signal_no_spikes: List[float]
        list contains coordinates of center of pressure for left foot"""
    signal_no_spikes = signal
    for index in range(1,len(signal)):
        if((signal.iloc[index-1]==0 and signal.iloc[index]>0 and signal.iloc[index+1]==0)\
                or (signal.iloc[index-1]==0 and signal.iloc[index]>0 and signal.iloc[index+1]>0) \
                or (signal.iloc[index-1]==0 and signal.iloc[index]<0 and signal.iloc[index+1]==0)):
            signal_no_spikes.iloc[index]=0
        else:
            pass
    return signal_no_spikes

In [7]:
def center_of_pressure_detection(file_position:int) -> Tuple[List[float],List[float],str]:

    """
    Parameters
    ----------
    file_name: pd.core.frame.DataFrame
        data frame contains time[s], measurements from 16 pressure sensors - 8 left leg and 8 right leg, and total force for left leg and right leg

    Returns
    -------
    y_left: List[float]
        list contains coordinates of center of pressure for left foot
    y_right: List[float]
        list contains coordinates of center of pressure for right foot
    """

    # L1 (-500,-800) R1 (500,-800)
    # L2 (-700,-400) R2 (700,-400)
    # L3 (-300,-400) R3 (300,-400)
    # L4 (-700,0)    R4 (700,0)
    # L5 (-300,0)    R5 (300,0)
    # L6 (-700,400)  R6 (700,400)
    # L7 (-300,400)  R7 (300,400)
    # L8 (-500,800)  R8 (500,800)
    
    # Open file 
    file_paths = []
    path_to_data_folder = os.path.join(os.getcwd(), "data")
#     print(os.path.join(path_to_data_folder, '*.txt'))
    for f in glob.glob(os.path.join(path_to_data_folder, '*.txt')):
        file_paths.append(f)
    #     print(f)
    fileName = os.path.splitext(os.path.basename(file_paths[file_position]))[0]
#     print(variable_name)
#     print(type(variable_name))
    variable_name = pd.read_csv(file_paths[file_position], delimiter='\t')
    variable_name.columns = ['Time[s]','VGRF-left1', 'VGRF-left2','VGRF-left3','VGRF-left4','VGRF-left5','VGRF-left6','VGRF-left7','VGRF-left8','VGRF-right1', 'VGRF-right2','VGRF-right3','VGRF-right4','VGRF-right5','VGRF-right6','VGRF-right7','VGRF-right8','Total Force-left','Total Force-right']

    y_coordinates = {'1': -800,'2': -400,'3': -400,'4': 0,'5': 0,'6': 400,'7': 400,'8': 800}
    filename_col_names_left_leg = ['VGRF-left1', 'VGRF-left2','VGRF-left3','VGRF-left4','VGRF-left5','VGRF-left6','VGRF-left7','VGRF-left8']
    filename_col_names_right_leg = ['VGRF-right1', 'VGRF-right2','VGRF-right3','VGRF-right4','VGRF-right5','VGRF-right6','VGRF-right7','VGRF-right8']
    y_coordinates_left = sum(variable_name[filename_col_names_left_leg[i]]*y_coordinates[str(i+1)] for i in range(len(filename_col_names_left_leg)))/variable_name['Total Force-left']
    y_coordinates_right = sum(variable_name[filename_col_names_right_leg[i]]*y_coordinates[str(i+1)] for i in range(len(filename_col_names_right_leg)))/variable_name['Total Force-right']

    # file_name['Total Force-left'] can be 0 because during swing phase total pressure equals to 0
    # then all NaN values are replaced with 0
    y_coordinates_left = y_coordinates_left.fillna(0)
    y_coordinates_right = y_coordinates_right.fillna(0)

    y_coordinates_left = remove_spikes(y_coordinates_left)
    y_coordinates_right = remove_spikes(y_coordinates_right)

    # filtering signals with medianfilter
    # y_coordinates_left_filtered = scipy.signal.medfilt(y_coordinates_left,kernel_size=31)
    # y_coordinates_right_filtered = scipy.signal.medfilt(y_coordinates_right,kernel_size=31)
    y_coordinates_left_filtered = scipy.ndimage.uniform_filter1d(y_coordinates_left,size=15)
    y_coordinates_right_filtered = scipy.ndimage.uniform_filter1d(y_coordinates_right,size=15)
    # y_coordinates_left_filtered = y_coordinates_left
    # y_coordinates_right_filtered = y_coordinates_right
    return y_coordinates_left_filtered, y_coordinates_right_filtered,fileName

In [ ]:
y_COP_left, y_COP_right,fileName = center_of_pressure_detection(0)

In [ ]:
print(fileName)

In [ ]:
plt.plot(y_COP_left[2500:3100])
plt.plot(y_COP_right[2500:3100])
plt.suptitle('Lokalizacja centrum nacisku na stopie podczas chodu  w zależności od czasu', size=13, y=1)
plt.xlabel("Czas [s]", fontdict={'fontsize': 11})
plt.ylabel("Lokalizacja centrum nacisku na stopie", fontdict={'fontsize': 11})

In [8]:
def binarize_CoP_signal(CoP_signal:np.ndarray)->List[int]:
    """
    Binarize signal of center of pressure during gait.
    Center of pressure is raising -> equals to 400 and stance phase
    Center of pressure is decreasign -> equals to 0 and swing phase
    
    Parameters
    ----------
    CoP_signal:np.ndarray
        signal which is calculated Center od Pressure during gait

    Returns
    -------
    binarized_CoP_singal: List[int]
        list contains two values only 400 means stance phase and 0 means swing phase
    """
        
    binarized_CoP_singal = []
    for i in range(1,len(CoP_signal)):
        if CoP_signal[i] - CoP_signal[i-1]>0:
            binarized_CoP_singal.append(400)
#         elif CoP_signal[i] - CoP_signal[i-1]<0 and CoP_signal[i-1] < 0:
#             binarized_CoP_singal.append(400)
        else:
            binarized_CoP_singal.append(0)
              
    return binarized_CoP_singal

In [ ]:
binarized_y_COP_left = binarize_CoP_signal(y_COP_left)
binarized_y_COP_right = binarize_CoP_signal(y_COP_right)

In [ ]:
plt.plot(y_COP_right[2500:3100])
plt.plot(binarized_y_COP_right[2500:3100])
plt.suptitle(" Centrum nacisku nacisku na stopie podczas chodu", size=13, y=1)
plt.legend(['Centrum nacisku','0 - faza przenoszenia, 400 - faza podparcia'], bbox_to_anchor =(1.25, 0.1))
plt.xlabel("Czas [s]", fontdict={'fontsize': 11})
plt.ylabel("Lokalizacja centrum nacisku na stopie", fontdict={'fontsize': 11})

In [10]:
import numpy as np

In [9]:
def sensor_placement_detection_in_gait_phases(signal: np.ndarray):
    """
    Parameters
    ----------
    signal: np.ndarray
        contains CoP data
    Returns
    -------

    """
    max_values_signal, _ = scipy.signal.find_peaks(signal,threshold=0,distance=70)
    min_values_signal, _ = scipy.signal.find_peaks(-signal,threshold=0,distance=70)

    # remove too low values of max peaks
    max_values_signal = [max_peak for max_peak in max_values_signal if signal[max_peak] > 100 ]
    min_values_signal = [min_peak for min_peak in min_values_signal if signal[min_peak] < -100 ]

    return max_values_signal, min_values_signal

In [ ]:
max_COP_left,min_COP_left = sensor_placement_detection_in_gait_phases(y_COP_left)

In [ ]:
plt.scatter(max_COP_left,y_COP_left[max_COP_left], marker='.', edgecolors='red')
plt.scatter(min_COP_left,y_COP_left[min_COP_left], marker='.')
plt.plot(y_COP_left)
plt.suptitle("Centrum nacisku na stopie podczas chodu", fontdict={'fontsize': 14})
plt.legend(['Centrum nacisku','Max','Min',], bbox_to_anchor =(1.5, 0.1))
plt.xlabel("Czas [s]", fontdict={'fontsize': 11})
plt.ylabel("Lokalizacja centrum nacisku na stopie", fontdict={'fontsize': 11})

In [ ]:
maximum_values = [i for i in max_COP_left if i<1200]
minimum_values = [i for i in min_COP_left if i<1200]
plt.scatter(maximum_values,y_COP_left[maximum_values], marker='.', edgecolors='red')
plt.scatter(minimum_values,y_COP_left[minimum_values], marker='.')
plt.plot(y_COP_left[0:1200])

In [11]:
def peaks_gait_mean_and_std_calc(peaks):
    return np.mean(peaks), np.std(peaks)

In [ ]:
mean_max_peaks_left, std_max_peaks_left= peaks_gait_mean_and_std_calc(y_COP_left[max_COP_left])
mean_min_peaks_left, std_min_peaks_left = peaks_gait_mean_and_std_calc(y_COP_left[min_COP_left])

In [ ]:
print(mean_max_peaks_left, std_max_peaks_left)
print(mean_min_peaks_left, std_min_peaks_left)

In [ ]:
print(max_COP_left[0])

In [12]:
import copy

In [13]:
def stance_swing_phase_detection(max_peaks_indices, min_peaks_indices):
    
    max_peaks_ind = copy.deepcopy(max_peaks_indices)
    min_peaks_ind = copy.deepcopy(min_peaks_indices)
    stance_duration = []
    swing_duration = []
    length_max_peaks_indices = len(max_peaks_ind)
    length_min_peaks_indices = len(min_peaks_ind)
#     print('length_max_peaks_ind',length_max_peaks_indices)
#     print('length_min_peaks_ind',length_min_peaks_indices)

    if length_min_peaks_indices > length_max_peaks_indices:
        length = length_max_peaks_indices
    else:
         length = length_min_peaks_indices

#     print('max_peaks_ind',max_peaks_ind)
#     print('min_peaks_ind',min_peaks_ind)
#     print('min_peaks_indices',min_peaks_indices)

    # check what is first -> heel strike (minimum) or toe-off (maximum)
    if max_peaks_indices[0]>min_peaks_indices[0]:
        for i in range(1,length):
            stance_time = (max_peaks_ind[i-1] - min_peaks_ind[i-1])*0.01
            if stance_time<1.5:
                stance_duration.append(stance_time)
                swing_duration.append((min_peaks_ind[i] - max_peaks_ind[i-1])*0.01)
            else:
                min_peaks_ind.pop(0)
    else:
        for i in range(1,length):
            stance_time = (max_peaks_ind[i] - min_peaks_ind[i-1])*0.01
            stance_duration.append(stance_time)
            swing_duration.append((min_peaks_ind[i-1] - max_peaks_ind[i-1])*0.01)
            if stance_time<1.5:
                stance_duration.append(stance_time)
                swing_duration.append((min_peaks_ind[i-1] - max_peaks_ind[i])*0.01)
            else:
                min_peaks_ind.pop(0)
    return stance_duration,swing_duration

In [ ]:
stance_left_duration,swin_left_duration = stance_swing_phase_detection(max_COP_left, min_COP_left)

In [ ]:
print(stance_left_duration)

In [ ]:
print(swin_left_duration)

In [ ]:
print(type(swin_left_duration))

In [14]:
def mean_phase_duration(phase_duration_list: List[float]) -> float:
    """
    Parameters
    ----------
    phase_duration: List[float]
        contains time duration of specifc phase <stance/swing>
    Returns
    -------
    mean_value_duration: float
    mean value of gait phase
    
    """
    
    mean_value_duration = np.mean(phase_duration_list)
    return mean_value_duration

def std_phase_duration(phase_duration_list: List[float]) -> float:
    """
    Parameters
    ----------
    phase_duration: List[float]
        contains time duration of specifc phase <stance/swing>
    Returns
    -------
    std_value_duration: float
    standard deviation value of gait phase
    
    """
    
    std_value_duration = np.std(phase_duration_list)
    return std_value_duration

In [ ]:
mean_value__stance_duration = mean_phase_duration(stance_left_duration)
mean_value__swing_duration = mean_phase_duration(swin_left_duration)
std_value__stance_duration = std_phase_duration(stance_left_duration)
std_value__swing_duration = std_phase_duration(swin_left_duration)

In [ ]:
print("mean_value__stance_duration: ", mean_value__stance_duration)
print("mean_value__swing_duration: ", mean_value__swing_duration)

In [ ]:
print("std_value__stance_duration: ", std_value__stance_duration)
print("std_value__swing_duration: ", std_value__swing_duration)

In [ ]:
    binarized_y_COP_left = binarize_CoP_signal(y_COP_left)
    binarized_y_COP_right = binarize_CoP_signal(y_COP_right)
    plt.plot(binarized_y_COP_left[0:200])
#     plt.plot(binarized_y_COP_right[0:200])

In [ ]:
print(binarized_y_COP_right)

In [20]:
def count_zeros(position, signal):
    zero_count = 0
#     print(position)
    print(len(signal))
    while signal[position]== 0:
        zero_count += 1
        position += 1
        print('position:',position)
    return position,zero_count

In [16]:
def remove_spikes_binarized_signal(binarized_y_COP_signal):
    
    indices_of_value_0 = []
    for idx, value in enumerate(binarized_y_COP_signal):
        if value == 0:
            indices_of_value_0.append(idx)
#     print(indices_of_value_0)
    
    # remove spikes -> if signal has spikes in value of 400 it means that this period of spikes should be whole as 400
    for i in range(1,len(indices_of_value_0)):
        if (indices_of_value_0[i] - indices_of_value_0[i-1])<20 and (indices_of_value_0[i] - indices_of_value_0[i-1])>1:
            list_of_400 = [400 for j in range(indices_of_value_0[i-1],indices_of_value_0[i]+1)]
#             print(list_of_400)
            binarized_y_COP_signal[indices_of_value_0[i-1]:(indices_of_value_0[i]+1)]=list_of_400
    
#     print(binarized_y_COP_signal)
    
    binarized_signal_corrected = copy.deepcopy(binarized_y_COP_signal)
    for j in range(1,len(binarized_signal_corrected)):
        if binarized_signal_corrected[j-1]*binarized_signal_corrected[j] == 0:
            position,zero_count = count_zeros(j,binarized_signal_corrected)
#             print(position)
            if zero_count < 10:
#                 print(j)
                list_of_400 = [400 for k in range(j,position+1)]
                binarized_signal_corrected[j:position+1]=list_of_400
                
    return binarized_y_COP_signal,binarized_signal_corrected

In [ ]:
binarized_y_COP_signal,binarized_signal_corrected_1 = remove_spikes_binarized_signal(binarized_y_COP_left)
plt.plot(binarized_y_COP_left[0:500])
plt.plot(y_COP_left[0:500])


In [ ]:
plt.plot(binarized_signal_corrected_1[0:500])
plt.plot(y_COP_left[0:500])

In [17]:
def double_support(y_COP_left,y_COP_right):
    
    double_support_list = []
    duration = 0
    binarized_y_COP_left = binarize_CoP_signal(y_COP_left)
    binarized_y_COP_signal_left, binarized_signal_corrected_final_left = remove_spikes_binarized_signal(binarized_y_COP_left)
    binarized_y_COP_right = binarize_CoP_signal(y_COP_right)
    binarized_y_COP_signal_right, binarized_signal_corrected_final_right = remove_spikes_binarized_signal(binarized_y_COP_right)
    
    if len(binarized_signal_corrected_final_right) < len(binarized_signal_corrected_final_left):
        while len(binarized_signal_corrected_final_right)!=0:
            cop_binary_right = binarized_signal_corrected_final_right.pop(0)
            cop_binary_left = binarized_signal_corrected_final_left.pop(0)
            if cop_binary_right == 400 and cop_binary_left == 400:
                duration += 1
            else:
                double_support_list.append(duration)
                duration = 0
        double_support_list = [x*0.01 for x in double_support_list if x!=0]
    else:
        while len(binarized_signal_corrected_final_left)!=0:
            cop_binary_right = binarized_signal_corrected_final_right.pop(0)
            cop_binary_left = binarized_signal_corrected_final_left.pop(0)
            if cop_binary_right == 400 and cop_binary_left == 400:
                duration += 1
            else:
                double_support_list.append(duration)
                duration = 0
        double_support_list = [x*0.01 for x in double_support_list if x!=0]
    return double_support_list

In [ ]:
double_support_list = double_support(y_COP_left,y_COP_right)
print(double_support_list)

In [ ]:
# Count number of files inside data dir

path_to_data_folder = os.path.join(os.getcwd(), "data")
file_count = 0
# Iterate directory
for path in os.listdir(path_to_data_folder):
    # check if current path is a file
    if os.path.isfile(os.path.join(path_to_data_folder, path)):
        file_count += 1
print('File count:', file_count)

df = pd.DataFrame()
for file_position in range(0,file_count):
    print(file_position)
    # Main loop
    y_COP_left, y_COP_right, filename = center_of_pressure_detection(file_position)

    # Left Leg
    max_COP_left, min_COP_left = sensor_placement_detection_in_gait_phases(y_COP_left)

    mean_max_peaks_left, std_max_peaks_left = peaks_gait_mean_and_std_calc(y_COP_left[max_COP_left])
    mean_min_peaks_left, std_min_peaks_left = peaks_gait_mean_and_std_calc(y_COP_left[min_COP_left])

    stance_left_duration,swing_left_duration = stance_swing_phase_detection(max_COP_left, min_COP_left)

    mean_value__stance_duration_left = mean_phase_duration(stance_left_duration)
    mean_value__swing_duration_left = mean_phase_duration(swing_left_duration)

    std_value__stance_duration_left = std_phase_duration(stance_left_duration)
    std_value__swing_duration_left = std_phase_duration(swing_left_duration)


    # Right Leg
    max_COP_right,min_COP_right = sensor_placement_detection_in_gait_phases(y_COP_right)

    mean_max_peaks_right, std_max_peaks_right = peaks_gait_mean_and_std_calc(y_COP_right[max_COP_right])
    mean_min_peaks_right, std_min_peaks_right = peaks_gait_mean_and_std_calc(y_COP_right[min_COP_right])

    stance_right_duration,swing_right_duration = stance_swing_phase_detection(max_COP_right, min_COP_right)

    mean_value__stance_duration_right = mean_phase_duration(stance_right_duration)
    mean_value__swing_duration_right = mean_phase_duration(swing_right_duration)

    std_value__stance_duration_right = std_phase_duration(stance_right_duration)
    std_value__swing_duration_right = std_phase_duration(swing_right_duration)


    # Left and right - doucle support
    double_suport_list = double_support(y_COP_left, y_COP_right)
    mean_double_support_duration = np.mean(double_suport_list)
    std_double_support_duration = np.std(double_suport_list)

    # Calc mean and std
    temp = pd.DataFrame(
        {
            'Patient': filename,
            'mean_max_peaks_left': mean_max_peaks_left,
            'std_max_peaks_left': std_max_peaks_left,
            'mean_min_peaks_left':mean_min_peaks_left,
            'std_min_peaks_left':std_min_peaks_left,
            'mean_value__stance_duration_left':mean_value__stance_duration_left,
            'std_value__stance_duration_left':std_value__stance_duration_left,
            'mean_value__swing_duration_left':mean_value__swing_duration_left,
            'std_value__swing_duration_left':std_value__swing_duration_left,
            'mean_max_peaks_right': mean_max_peaks_right,
            'std_max_peaks_right': std_max_peaks_right,
            'mean_min_peaks_right':mean_min_peaks_right,
            'std_min_peaks_right':std_min_peaks_right,
            'mean_value__stance_duration_right':mean_value__stance_duration_right,
            'std_value__stance_duration_right':std_value__stance_duration_right,
            'mean_value__swing_duration_right':mean_value__swing_duration_right,
            'std_value__swing_duration_right':std_value__swing_duration_right,
            'mean_double_support_duration':mean_double_support_duration,
            'std_double_support_duration':std_double_support_duration 
        }, index=[file_position]
    )
    df = pd.concat([df, temp])

In [ ]:
df.to_csv('patient_data_param.csv', index=False)

In [ ]:
file_position = 0
y_COP_left, y_COP_right, filename = center_of_pressure_detection(file_position)

# Left Leg
max_COP_left, min_COP_left = sensor_placement_detection_in_gait_phases(y_COP_left)

mean_max_peaks_left, std_max_peaks_left = peaks_gait_mean_and_std_calc(y_COP_left[max_COP_left])
mean_min_peaks_left, std_min_peaks_left = peaks_gait_mean_and_std_calc(y_COP_left[min_COP_left])

stance_left_duration,swing_left_duration = stance_swing_phase_detection(max_COP_left, min_COP_left)

mean_value__stance_duration_left = mean_phase_duration(stance_left_duration)
mean_value__swing_duration_left = mean_phase_duration(swing_left_duration)

std_value__stance_duration_left = std_phase_duration(stance_left_duration)
std_value__swing_duration_left = std_phase_duration(swing_left_duration)


# Right Leg
max_COP_right,min_COP_right = sensor_placement_detection_in_gait_phases(y_COP_right)

mean_max_peaks_right, std_max_peaks_right = peaks_gait_mean_and_std_calc(y_COP_right[max_COP_right])
mean_min_peaks_right, std_min_peaks_right = peaks_gait_mean_and_std_calc(y_COP_right[min_COP_right])

stance_right_duration,swing_right_duration = stance_swing_phase_detection(max_COP_right, min_COP_right)

mean_value__stance_duration_right = mean_phase_duration(stance_right_duration)
mean_value__swing_duration_right = mean_phase_duration(swing_right_duration)

std_value__stance_duration_right = std_phase_duration(stance_right_duration)
std_value__swing_duration_right = std_phase_duration(swing_right_duration)


# Left and right - doucle support
double_suport_list = double_support(y_COP_left, y_COP_right)
mean_double_support_duration = np.mean(double_suport_list)
std_double_support_duration = np.std(double_suport_list)

# Calc mean and std
temp = pd.DataFrame(
    {
        'Patient': filename,
        'mean_max_peaks_left': mean_max_peaks_left,
        'std_max_peaks_left': std_max_peaks_left,
        'mean_min_peaks_left':mean_min_peaks_left,
        'std_min_peaks_left':std_min_peaks_left,
        'mean_value__stance_duration_left':mean_value__stance_duration_left,
        'std_value__stance_duration_left':std_value__stance_duration_left,
        'mean_value__swing_duration_left':mean_value__swing_duration_left,
        'std_value__swing_duration_left':std_value__swing_duration_left,
        'mean_max_peaks_right': mean_max_peaks_right,
        'std_max_peaks_right': std_max_peaks_right,
        'mean_min_peaks_right':mean_min_peaks_right,
        'std_min_peaks_right':std_min_peaks_right,
        'mean_value__stance_duration_right':mean_value__stance_duration_right,
        'std_value__stance_duration_right':std_value__stance_duration_right,
        'mean_value__swing_duration_right':mean_value__swing_duration_right,
        'std_value__swing_duration_right':std_value__swing_duration_right,
        'mean_double_support_duration':mean_double_support_duration,
        'std_double_support_duration':std_double_support_duration 
    }, index=[file_position]
)
df = pd.concat([df, temp])

In [ ]:
file_position = 1
y_COP_left, y_COP_right, filename = center_of_pressure_detection(file_position)

# Left Leg
max_COP_left, min_COP_left = sensor_placement_detection_in_gait_phases(y_COP_left)

mean_max_peaks_left, std_max_peaks_left = peaks_gait_mean_and_std_calc(y_COP_left[max_COP_left])
mean_min_peaks_left, std_min_peaks_left = peaks_gait_mean_and_std_calc(y_COP_left[min_COP_left])

stance_left_duration,swing_left_duration = stance_swing_phase_detection(max_COP_left, min_COP_left)

mean_value__stance_duration_left = mean_phase_duration(stance_left_duration)
mean_value__swing_duration_left = mean_phase_duration(swing_left_duration)

std_value__stance_duration_left = std_phase_duration(stance_left_duration)
std_value__swing_duration_left = std_phase_duration(swing_left_duration)


# Right Leg
max_COP_right,min_COP_right = sensor_placement_detection_in_gait_phases(y_COP_right)

mean_max_peaks_right, std_max_peaks_right = peaks_gait_mean_and_std_calc(y_COP_right[max_COP_right])
mean_min_peaks_right, std_min_peaks_right = peaks_gait_mean_and_std_calc(y_COP_right[min_COP_right])

stance_right_duration,swing_right_duration = stance_swing_phase_detection(max_COP_right, min_COP_right)

mean_value__stance_duration_right = mean_phase_duration(stance_right_duration)
mean_value__swing_duration_right = mean_phase_duration(swing_right_duration)

std_value__stance_duration_right = std_phase_duration(stance_right_duration)
std_value__swing_duration_right = std_phase_duration(swing_right_duration)


# Left and right - doucle support
double_suport_list = double_support(y_COP_left, y_COP_right)
mean_double_support_duration = np.mean(double_suport_list)
std_double_support_duration = np.std(double_suport_list)

# Calc mean and std
temp = pd.DataFrame(
    {
        'Patient': filename,
        'mean_max_peaks_left': mean_max_peaks_left,
        'std_max_peaks_left': std_max_peaks_left,
        'mean_min_peaks_left':mean_min_peaks_left,
        'std_min_peaks_left':std_min_peaks_left,
        'mean_value__stance_duration_left':mean_value__stance_duration_left,
        'std_value__stance_duration_left':std_value__stance_duration_left,
        'mean_value__swing_duration_left':mean_value__swing_duration_left,
        'std_value__swing_duration_left':std_value__swing_duration_left,
        'mean_max_peaks_right': mean_max_peaks_right,
        'std_max_peaks_right': std_max_peaks_right,
        'mean_min_peaks_right':mean_min_peaks_right,
        'std_min_peaks_right':std_min_peaks_right,
        'mean_value__stance_duration_right':mean_value__stance_duration_right,
        'std_value__stance_duration_right':std_value__stance_duration_right,
        'mean_value__swing_duration_right':mean_value__swing_duration_right,
        'std_value__swing_duration_right':std_value__swing_duration_right,
        'mean_double_support_duration':mean_double_support_duration,
        'std_double_support_duration':std_double_support_duration 
    }, index=[file_position]
)
df = pd.concat([df, temp])

In [21]:
file_position = 2
y_COP_left, y_COP_right, filename = center_of_pressure_detection(file_position)

# Left Leg
max_COP_left, min_COP_left = sensor_placement_detection_in_gait_phases(y_COP_left)

mean_max_peaks_left, std_max_peaks_left = peaks_gait_mean_and_std_calc(y_COP_left[max_COP_left])
mean_min_peaks_left, std_min_peaks_left = peaks_gait_mean_and_std_calc(y_COP_left[min_COP_left])

stance_left_duration,swing_left_duration = stance_swing_phase_detection(max_COP_left, min_COP_left)

mean_value__stance_duration_left = mean_phase_duration(stance_left_duration)
mean_value__swing_duration_left = mean_phase_duration(swing_left_duration)

std_value__stance_duration_left = std_phase_duration(stance_left_duration)
std_value__swing_duration_left = std_phase_duration(swing_left_duration)


# Right Leg
max_COP_right,min_COP_right = sensor_placement_detection_in_gait_phases(y_COP_right)

mean_max_peaks_right, std_max_peaks_right = peaks_gait_mean_and_std_calc(y_COP_right[max_COP_right])
mean_min_peaks_right, std_min_peaks_right = peaks_gait_mean_and_std_calc(y_COP_right[min_COP_right])

stance_right_duration,swing_right_duration = stance_swing_phase_detection(max_COP_right, min_COP_right)

mean_value__stance_duration_right = mean_phase_duration(stance_right_duration)
mean_value__swing_duration_right = mean_phase_duration(swing_right_duration)

std_value__stance_duration_right = std_phase_duration(stance_right_duration)
std_value__swing_duration_right = std_phase_duration(swing_right_duration)


# Left and right - doucle support
double_suport_list = double_support(y_COP_left, y_COP_right)
mean_double_support_duration = np.mean(double_suport_list)
std_double_support_duration = np.std(double_suport_list)

# Calc mean and std
temp = pd.DataFrame(
    {
        'Patient': filename,
        'mean_max_peaks_left': mean_max_peaks_left,
        'std_max_peaks_left': std_max_peaks_left,
        'mean_min_peaks_left':mean_min_peaks_left,
        'std_min_peaks_left':std_min_peaks_left,
        'mean_value__stance_duration_left':mean_value__stance_duration_left,
        'std_value__stance_duration_left':std_value__stance_duration_left,
        'mean_value__swing_duration_left':mean_value__swing_duration_left,
        'std_value__swing_duration_left':std_value__swing_duration_left,
        'mean_max_peaks_right': mean_max_peaks_right,
        'std_max_peaks_right': std_max_peaks_right,
        'mean_min_peaks_right':mean_min_peaks_right,
        'std_min_peaks_right':std_min_peaks_right,
        'mean_value__stance_duration_right':mean_value__stance_duration_right,
        'std_value__stance_duration_right':std_value__stance_duration_right,
        'mean_value__swing_duration_right':mean_value__swing_duration_right,
        'std_value__swing_duration_right':std_value__swing_duration_right,
        'mean_double_support_duration':mean_double_support_duration,
        'std_double_support_duration':std_double_support_duration 
    }, index=[file_position]
)
df = pd.concat([df, temp])

12117
position: 2
position: 3
position: 4
position: 5
position: 6
position: 7
position: 8
position: 9
position: 10
position: 11
position: 12
position: 13
position: 14
12117
position: 3
position: 4
position: 5
position: 6
position: 7
position: 8
position: 9
position: 10
position: 11
position: 12
position: 13
position: 14
12117
position: 4
position: 5
position: 6
position: 7
position: 8
position: 9
position: 10
position: 11
position: 12
position: 13
position: 14
12117
position: 5
position: 6
position: 7
position: 8
position: 9
position: 10
position: 11
position: 12
position: 13
position: 14
12117
position: 6
position: 7
position: 8
position: 9
position: 10
position: 11
position: 12
position: 13
position: 14
12117
position: 71
position: 72
12117
position: 76
position: 77
position: 78
position: 79
position: 80
position: 81
position: 82
position: 83
position: 84
position: 85
position: 86
position: 87
position: 88
position: 89
position: 90
position: 91
position: 92
position: 93
position: 94


position: 115
position: 116
position: 117
position: 118
position: 119
position: 120
position: 121
position: 122
12117
position: 104
position: 105
position: 106
position: 107
position: 108
position: 109
position: 110
position: 111
position: 112
position: 113
position: 114
position: 115
position: 116
position: 117
position: 118
position: 119
position: 120
position: 121
position: 122
12117
position: 105
position: 106
position: 107
position: 108
position: 109
position: 110
position: 111
position: 112
position: 113
position: 114
position: 115
position: 116
position: 117
position: 118
position: 119
position: 120
position: 121
position: 122
12117
position: 106
position: 107
position: 108
position: 109
position: 110
position: 111
position: 112
position: 113
position: 114
position: 115
position: 116
position: 117
position: 118
position: 119
position: 120
position: 121
position: 122
12117
position: 107
position: 108
position: 109
position: 110
position: 111
position: 112
position: 113
position: 

position: 205
position: 206
position: 207
position: 208
position: 209
position: 210
position: 211
position: 212
position: 213
position: 214
position: 215
position: 216
position: 217
position: 218
position: 219
position: 220
position: 221
position: 222
position: 223
position: 224
position: 225
position: 226
12117
position: 196
position: 197
position: 198
position: 199
position: 200
position: 201
position: 202
position: 203
position: 204
position: 205
position: 206
position: 207
position: 208
position: 209
position: 210
position: 211
position: 212
position: 213
position: 214
position: 215
position: 216
position: 217
position: 218
position: 219
position: 220
position: 221
position: 222
position: 223
position: 224
position: 225
position: 226
12117
position: 197
position: 198
position: 199
position: 200
position: 201
position: 202
position: 203
position: 204
position: 205
position: 206
position: 207
position: 208
position: 209
position: 210
position: 211
position: 212
position: 213
position

position: 306
position: 307
position: 308
position: 309
position: 310
position: 311
position: 312
position: 313
position: 314
position: 315
position: 316
position: 317
position: 318
position: 319
position: 320
position: 321
position: 322
position: 323
position: 324
position: 325
position: 326
position: 327
position: 328
position: 329
position: 330
position: 331
position: 332
position: 333
12117
position: 295
position: 296
position: 297
position: 298
position: 299
position: 300
position: 301
position: 302
position: 303
position: 304
position: 305
position: 306
position: 307
position: 308
position: 309
position: 310
position: 311
position: 312
position: 313
position: 314
position: 315
position: 316
position: 317
position: 318
position: 319
position: 320
position: 321
position: 322
position: 323
position: 324
position: 325
position: 326
position: 327
position: 328
position: 329
position: 330
position: 331
position: 332
position: 333
12117
position: 296
position: 297
position: 298
position

position: 417
position: 418
position: 419
position: 420
position: 421
position: 422
position: 423
position: 424
position: 425
position: 426
position: 427
position: 428
position: 429
position: 430
position: 431
position: 432
position: 433
position: 434
position: 435
position: 436
position: 437
12117
position: 391
position: 392
position: 393
position: 394
position: 395
position: 396
position: 397
position: 398
position: 399
position: 400
position: 401
position: 402
position: 403
position: 404
position: 405
position: 406
position: 407
position: 408
position: 409
position: 410
position: 411
position: 412
position: 413
position: 414
position: 415
position: 416
position: 417
position: 418
position: 419
position: 420
position: 421
position: 422
position: 423
position: 424
position: 425
position: 426
position: 427
position: 428
position: 429
position: 430
position: 431
position: 432
position: 433
position: 434
position: 435
position: 436
position: 437
12117
position: 392
position: 393
position

...

position: 11798
position: 11799
position: 11800
position: 11801
position: 11802
position: 11803
position: 11804
position: 11805
position: 11806
position: 11807
position: 11808
position: 11809
position: 11810
position: 11811
position: 11812
position: 11813
position: 11814
position: 11815
12117
position: 11786
position: 11787
position: 11788
position: 11789
position: 11790
position: 11791
position: 11792
position: 11793
position: 11794
position: 11795
position: 11796
position: 11797
position: 11798
position: 11799
position: 11800
position: 11801
position: 11802
position: 11803
position: 11804
position: 11805
position: 11806
position: 11807
position: 11808
position: 11809
position: 11810
position: 11811
position: 11812
position: 11813
position: 11814
position: 11815
12117
position: 11787
position: 11788
position: 11789
position: 11790
position: 11791
position: 11792
position: 11793
position: 11794
position: 11795
position: 11796
position: 11797
position: 11798
position: 11799
position: 11

position: 11912
position: 11913
position: 11914
position: 11915
position: 11916
position: 11917
position: 11918
position: 11919
position: 11920
12117
position: 11881
position: 11882
position: 11883
position: 11884
position: 11885
position: 11886
position: 11887
position: 11888
position: 11889
position: 11890
position: 11891
position: 11892
position: 11893
position: 11894
position: 11895
position: 11896
position: 11897
position: 11898
position: 11899
position: 11900
position: 11901
position: 11902
position: 11903
position: 11904
position: 11905
position: 11906
position: 11907
position: 11908
position: 11909
position: 11910
position: 11911
position: 11912
position: 11913
position: 11914
position: 11915
position: 11916
position: 11917
position: 11918
position: 11919
position: 11920
12117
position: 11882
position: 11883
position: 11884
position: 11885
position: 11886
position: 11887
position: 11888
position: 11889
position: 11890
position: 11891
position: 11892
position: 11893
position: 11

position: 12005
position: 12006
position: 12007
position: 12008
position: 12009
position: 12010
position: 12011
position: 12012
position: 12013
position: 12014
position: 12015
position: 12016
position: 12017
position: 12018
position: 12019
position: 12020
position: 12021
position: 12022
position: 12023
position: 12024
12117
position: 11979
position: 11980
position: 11981
position: 11982
position: 11983
position: 11984
position: 11985
position: 11986
position: 11987
position: 11988
position: 11989
position: 11990
position: 11991
position: 11992
position: 11993
position: 11994
position: 11995
position: 11996
position: 11997
position: 11998
position: 11999
position: 12000
position: 12001
position: 12002
position: 12003
position: 12004
position: 12005
position: 12006
position: 12007
position: 12008
position: 12009
position: 12010
position: 12011
position: 12012
position: 12013
position: 12014
position: 12015
position: 12016
position: 12017
position: 12018
position: 12019
position: 12020
po

position: 12016
position: 12017
position: 12018
position: 12019
position: 12020
position: 12021
position: 12022
position: 12023
position: 12024
12117
position: 12016
position: 12017
position: 12018
position: 12019
position: 12020
position: 12021
position: 12022
position: 12023
position: 12024
12117
position: 12078
position: 12079
position: 12080
position: 12081
position: 12082
position: 12083
position: 12084
position: 12085
position: 12086
position: 12087
position: 12088
position: 12089
position: 12090
position: 12091
position: 12092
position: 12093
position: 12094
position: 12095
position: 12096
position: 12097
position: 12098
position: 12099
position: 12100
position: 12101
position: 12102
position: 12103
position: 12104
position: 12105
position: 12106
position: 12107
position: 12108
position: 12109
position: 12110
position: 12111
position: 12112
position: 12113
position: 12114
position: 12115
position: 12116
position: 12117


IndexError: list index out of range